In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as sts
import statsmodels.api as sm
import warnings


from statsmodels.tsa.stattools import acf
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats.mstats import winsorize
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from scipy.stats import bartlett
from scipy.stats import levene
from sqlalchemy import create_engine
from sklearn import linear_model

warnings.filterwarnings('ignore')

### 1İNCİ ÖDEV

In [2]:
hava=pd.read_csv(".\\WeatherHistory.csv")

In [4]:
hava["fark"]=hava["Temperature (C)"]-hava["Apparent Temperature (C)"]

In [5]:
Y=hava["fark"]
X=hava[["Humidity", "Wind Speed (km/h)"]]

X=sm.add_constant(X)

sonuc=sm.OLS(Y, X).fit()
sonuc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   fark   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Sat, 20 Apr 2019   Prob (F-statistic):               0.00
Time:                        07:08:54   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -2.4381      0.021   -115.948      0.000      -2.479      -2.397
Humidity              3.0292      0.024    126.479      0.000       2.982       3.076
Wind Speed (km/h)     0.1193      0.001    176.164      0.000       0.118       0.121
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.264
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                           0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R-kare ve ayarlanmış R-kare değerleri 0.288 'dir. Yani, modelimiz toplam varyansın sadece % 28.8 'ini açıklıyor. Oldukça düşük bir oran ve işe yaramaz. Modelimizi biraz geliştirmeye ihtiyacımız var.

In [6]:
hava["Hum_Wind"]=hava["Humidity"]*hava["Wind Speed (km/h)"]

Y=hava["fark"]
X=hava[["Humidity", "Wind Speed (km/h)", "Hum_Wind"]]

X=sm.add_constant(X)

sonuc=sm.OLS(Y, X).fit()
sonuc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   fark   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Sat, 20 Apr 2019   Prob (F-statistic):               0.00
Time:                        07:18:56   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.0839      0.033     -2.511      0.012      -0.149      -0.018
Humidity             -0.1775      0.043     -4.133      0.000      -0.262      -0.093
Wind Speed (km/h)    -0.0905      0.002    -36.797      0.000      -0.095      -0.086
Hum_Wind              0.2971      0.003     88.470      0.000       0.291       0.304
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.262
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                           0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Etkileşimle beraber varyansı açıklama oranımız % 34.1 'e yükseldi.

In [9]:
Y=hava["fark"]
X=hava[["Humidity", "Wind Speed (km/h)","Apparent Temperature (C)", "Hum_Wind"]]

X=sm.add_constant(X)

sonuc=sm.OLS(Y, X).fit()
sonuc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   fark   R-squared:                       0.726
Model:                            OLS   Adj. R-squared:                  0.726
Method:                 Least Squares   F-statistic:                 6.377e+04
Date:                Sat, 20 Apr 2019   Prob (F-statistic):               0.00
Time:                        07:23:06   Log-Likelihood:            -1.2445e+05
No. Observations:               96453   AIC:                         2.489e+05
Df Residuals:                   96448   BIC:                         2.490e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        4.8279      0.025    190.273      0.000       4.778       4.878
Humidity                    -4.4221      0.030   -147.279      0.000      -4.481      -4.363
Wind Speed (km/h)           -0.1156      0.002    -72.758      0.000      -0.119      -0.112
Apparent Temperature (C)    -0.1259      0.000   -367.582      0.000      -0.127      -0.125
Hum_Wind                     0.2768      0.002    127.674      0.000       0.273       0.281
==============================================================================
Omnibus:                     5823.257   Durbin-Watson:                   0.451
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8780.618
Skew:                          -0.516   Prob(JB):                         0.00
Kurtosis:                       4.058   Cond. No.                         265.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Açıklanan varyans % 72.6 'ya çıktı. Daha tatminkar bir sonuç  elde ettik.

- Etkileşim teriminin modele katkısı % 5.3,
- Görünür sıcaklık teriminin modele katkısı % 38.5 'dir.

Görünür sıcaklık, tek başına diğer üç terimden daha fazla varyans açıklamaktadır.

##### 1inci Model
- AIC : 340900
- BIC : 340900

##### 2nci Model
- AIC : 333400
- BIC : 333400

##### 3ncü Model
- AIC : 248900
- BIC : 248900

Üç modelimiz içerisinde, AIC ve BIC değerleri en düşük olan 3üncü modeldir. Bu durumda, en iyi model 3üncü modeldir.

### 2NCİ ÖDEV

In [10]:
knt=pd.read_csv(".\\konut_train.csv")

kategorik=[i for i in knt.columns if knt[i].dtype=="object"]
surekli=[i for i in knt.columns if (knt[i].dtype=="int64" or knt[i].dtype=="float64")]

konut=knt
for i in kategorik:
    konut[i]=knt[i].fillna("Null")
for i in surekli:
    konut[i]=knt[i].fillna(0)

In [11]:
Y=konut["SalePrice"]
X=konut[["YearBuilt", "TotalBsmtSF", "TotRmsAbvGrd", "LotFrontage"]]

X=sm.add_constant(X)
sonuc=sm.OLS(Y, X).fit()
sonuc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.616
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     582.6
Date:                Sat, 20 Apr 2019   Prob (F-statistic):          3.59e-300
Time:                        07:44:02   Log-Likelihood:                -17846.
No. Observations:                1460   AIC:                         3.570e+04
Df Residuals:                    1455   BIC:                         3.573e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.795e+06   9.07e+04    -19.779      0.000   -1.97e+06   -1.62e+06
YearBuilt      902.1209     46.553     19.378      0.000     810.803     993.439
TotalBsmtSF     65.4727      3.388     19.322      0.000      58.826      72.119
TotRmsAbvGrd  1.918e+04    840.426     22.819      0.000    1.75e+04    2.08e+04
LotFrontage     54.9156     38.957      1.410      0.159     -21.503     131.334
==============================================================================
Omnibus:                      412.541   Durbin-Watson:                   1.917
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14425.262
Skew:                           0.621   Prob(JB):                         0.00
Kurtosis:                      18.349   Cond. No.                     1.58e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

###### 1İNCİ MODEL:
- F-Testi    : 582.6
- P değeri   : 3.59e-300
- R-kare     : 0.616
- Ayr.R-kare : 0.615
- AIC        : 35700
- BIC        : 25730

##### Değerleri yorumlarsak;
- F-Testi ve P değerine baktığımızda, modelimizin boş modele göre daha iyi çalıştığı ve girilen değişkenlerin modelimize bir şeyler kattığunı söyleyebiliriz.
- R-kare ve ayarlanmış R-kare testleri, bize modelimizin toplam varyansın % 61.5 'ini açıkladığını göstermektedir.

In [77]:
karsilastirma=pd.DataFrame(columns={"parametre_sayisi", "F_test", "p_degeri", "R2", "ayr_R2", "AIC", "BIC"})

In [78]:
karsilastirma=karsilastirma.append({ "parametre_sayisi":4,
                                     "F_test":582.6,
                                     "p_degeri":0.0,
                                     "R2":0.616,
                                     "ayr_R2":0.615,
                                     "AIC":35700,
                                     "BIC":25730}, 
                                     ignore_index=True)

In [12]:
Y=konut["SalePrice"]
X=konut[["OverallQual", "TotalBsmtSF", "1stFlrSF", "GrLivArea"]]

X=sm.add_constant(X)
sonuc=sm.OLS(Y, X).fit()
sonuc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.743
Method:                 Least Squares   F-statistic:                     1057.
Date:                Sat, 20 Apr 2019   Prob (F-statistic):               0.00
Time:                        07:56:16   Log-Likelihood:                -17550.
No. Observations:                1460   AIC:                         3.511e+04
Df Residuals:                    1455   BIC:                         3.514e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -1.09e+05   5042.479    -21.613      0.000   -1.19e+05   -9.91e+04
OverallQual  2.846e+04   1028.503     27.671      0.000    2.64e+04    3.05e+04
TotalBsmtSF    25.7604      4.453      5.784      0.000      17.025      34.496
1stFlrSF       16.6766      5.181      3.219      0.001       6.513      26.840
GrLivArea      45.9858      2.761     16.654      0.000      40.569      51.402
==============================================================================
Omnibus:                      511.104   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            43306.711
Skew:                          -0.683   Prob(JB):                         0.00
Kurtosis:                      29.646   Cond. No.                     1.10e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.1e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

###### 2NCİ MODEL:
- F-Testi    : 1057
- P değeri   : 0.0
- R-kare     : 0.744
- Ayr.R-kare : 0.743
- AIC        : 35110
- BIC        : 35140

##### Değerleri yorumlarsak;
- F-Testi ve P değerine baktığımızda, modelimizin boş modele göre daha iyi çalıştığı ve girilen değişkenlerin modelimize bir şeyler kattığunı söyleyebiliriz.
- R-kare ve ayarlanmış R-kare testleri, bize modelimizin toplam varyansın % 74.3 'ini açıkladığını göstermektedir.

In [79]:
karsilastirma=karsilastirma.append({ "parametre_sayisi":4,
                                     "F_test":1057,
                                     "p_degeri":0.0,
                                     "R2":0.744,
                                     "ayr_R2":0.743,
                                     "AIC":35110,
                                     "BIC":35140}, 
                                     ignore_index=True)

In [16]:
Y=konut["SalePrice"]
X=konut[["OverallQual", "GarageCars", "YearBuilt", "GrLivArea", "FullBath", "Fireplaces"]]

X=sm.add_constant(X)
sonuc=sm.OLS(Y, X).fit()
sonuc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.758
Model:                            OLS   Adj. R-squared:                  0.757
Method:                 Least Squares   F-statistic:                     756.9
Date:                Sat, 20 Apr 2019   Prob (F-statistic):               0.00
Time:                        08:05:38   Log-Likelihood:                -17509.
No. Observations:                1460   AIC:                         3.503e+04
Df Residuals:                    1453   BIC:                         3.507e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -8.985e+05   9.13e+04     -9.846      0.000   -1.08e+06   -7.19e+05
OverallQual  2.197e+04   1171.412     18.753      0.000    1.97e+04    2.43e+04
GarageCars   1.566e+04   1861.507      8.413      0.000     1.2e+04    1.93e+04
YearBuilt     424.7097     48.051      8.839      0.000     330.453     518.966
GrLivArea      56.2999      3.113     18.086      0.000      50.193      62.406
FullBath    -7300.4099   2707.560     -2.696      0.007   -1.26e+04   -1989.266
Fireplaces   1.078e+04   1838.608      5.860      0.000    7168.433    1.44e+04
==============================================================================
Omnibus:                      447.368   Durbin-Watson:                   1.997
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16284.619
Skew:                           0.731   Prob(JB):                         0.00
Kurtosis:                      19.296   Cond. No.                     2.23e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.23e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

###### 3ÜNCÜ MODEL:
- F-Testi    : 756.9
- P değeri   : 0.0
- R-kare     : 0.758
- Ayr.R-kare : 0.757
- AIC        : 35030
- BIC        : 35070

##### Değerleri yorumlarsak;
- F-Testi ve P değerine baktığımızda, modelimizin boş modele göre daha iyi çalıştığı ve girilen değişkenlerin modelimize bir şeyler kattığunı söyleyebiliriz.
- R-kare ve ayarlanmış R-kare testleri, bize modelimizin toplam varyansın % 75.7 'sini açıkladığını göstermektedir.

In [80]:
karsilastirma=karsilastirma.append({ "parametre_sayisi":6,
                                     "F_test":756.9,
                                     "p_degeri":0.0,
                                     "R2":0.758,
                                     "ayr_R2":0.757,
                                     "AIC":35030,
                                     "BIC":35030}, 
                                     ignore_index=True)

In [37]:
konut["LivingArea"]=konut["GrLivArea"] * konut["MasVnrArea"]

Y=konut["SalePrice"]
X=konut[["OverallQual", "TotalBsmtSF", "1stFlrSF", "GrLivArea", "LivingArea"]]

X=sm.add_constant(X)
sonuc=sm.OLS(Y, X).fit()
sonuc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.750
Method:                 Least Squares   F-statistic:                     876.4
Date:                Sat, 20 Apr 2019   Prob (F-statistic):               0.00
Time:                        08:21:27   Log-Likelihood:                -17530.
No. Observations:                1460   AIC:                         3.507e+04
Df Residuals:                    1454   BIC:                         3.510e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -9.682e+04   5328.786    -18.168      0.000   -1.07e+05   -8.64e+04
OverallQual  2.786e+04   1019.185     27.332      0.000    2.59e+04    2.99e+04
TotalBsmtSF    21.6000      4.442      4.862      0.000      12.886      30.314
1stFlrSF       17.6683      5.114      3.455      0.001       7.636      27.700
GrLivArea      40.1272      2.875     13.956      0.000      34.487      45.767
LivingArea      0.0188      0.003      6.374      0.000       0.013       0.025
==============================================================================
Omnibus:                      733.244   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            73675.082
Skew:                          -1.385   Prob(JB):                         0.00
Kurtosis:                      37.690   Cond. No.                     2.42e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.42e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

###### 4ÜNCÜ MODEL:
- F-Testi    : 876.4
- P değeri   : 0.0
- R-kare     : 0.751
- Ayr.R-kare : 0.750
- AIC        : 35070
- BIC        : 35100

##### Değerleri yorumlarsak;
- F-Testi ve P değerine baktığımızda, modelimizin boş modele göre daha iyi çalıştığı ve girilen değişkenlerin modelimize bir şeyler kattığunı söyleyebiliriz.
- R-kare ve ayarlanmış R-kare testleri, bize modelimizin toplam varyansın % 75 'ini açıkladığını göstermektedir.

In [81]:
karsilastirma=karsilastirma.append({ "parametre_sayisi":5,
                                     "F_test":876.4,
                                     "p_degeri":0.0,
                                     "R2":0.751,
                                     "ayr_R2":0.750,
                                     "AIC":35070,
                                     "BIC":35100}, 
                                     ignore_index=True)

In [82]:
karsilastirma.T

,0,1,2,3
ayr_R2,0.615,0.743,0.757,0.750
F_test,582.600,1057.000,756.900,876.400
parametre_sayisi,4.000,4.000,6.000,5.000
R2,0.616,0.744,0.758,0.751
BIC,25730.000,35140.000,35030.000,35100.000
AIC,35700.000,35110.000,35030.000,35070.000
p_degeri,0.000,0.000,0.000,0.000


##### KARŞILAŞTIRMA:
- 1inci modelimiz en kötü modeldir.
- F-testi açısından en iyi model 2nci modeldir.
- R-kare ve ayarlanmış R-kare değerleri açısından en iyi model 3üncü modeldir.
- AIC açısından en iyi model 3üncü modeldir.
- BIC açısından en iyi model 3üncü modeldir.

Ayrıca, parametre sayıları açısından en az parametre 2nci modelde, en çok 3üncü modeldedir.

Tüm kriterler açısından bakıldığında % 75 varyans oranı ile 4üncü model tercih edilebilir.